In [1]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import re
from sklearn.externals import joblib
from sklearn.model_selection import GroupShuffleSplit
%matplotlib inline

In [2]:
data = pd.read_csv("keiba.csv",encoding="shift-jis",na_values="")
print("データの形")
print(data.shape)
data.head()

データの形
(29918, 12)


,racekey,horse_number,horse_name,order_of_finish,popularity,horse_weight,tanshou_odds,hukushou_odds,tanshou_odds_at10,hukushou_odds_at10,IDM,hot_code
0,216220101,1410370320160710,マイネルトルペード,2,4,412,15.8,1.3,14.4,1.4,31,3
1,216220102,1410026420160710,アントレーン,6,2,418,6.9,1,6.4,1.1,18,3
2,216220103,1410432120160710,ジャストヒーロー,8,9,458,145.4,10,58.3,10.1,11,3
3,216220104,1410141620160710,サツキチャン,7,5,408,19.8,1.5,16.4,1.4,15,3
4,216220105,1410515620160710,サクセスムーン,4,8,442,102.3,7.9,39.1,5.4,29,3


In [3]:
#raceのみを表すID
data.loc[:,"raceid"] = data.loc[:,"racekey"].str[:7]

In [4]:
data = data[data["order_of_finish"].str.contains("(\d)+")]
data = data[data["popularity"].str.contains("(\d)+")]
data = data[data["tanshou_odds"].str.contains("(\d)+")]
data = data[data["IDM"].str.contains("(\d)+")]
data = data[data["hot_code"].str.contains("(\d)+")]
data = data[data["tanshou_odds_at10"].str.contains("^(\d)+(?!.*\s).*$")]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  """
C:\ProgramData\Anaconda3\lib\site-

In [5]:
data.loc[:,"order_of_finish_num"] = data.loc[:,"order_of_finish"].astype(int)
data.loc[:,"popularity_num"] = data.loc[:,"popularity"].astype(int)
data.loc[:,"horse_weight_num"] = data.loc[:,"horse_weight"].astype(int)
data.loc[:,"tanshou_odds_num"] = data.loc[:,"tanshou_odds"].astype(float)
data.loc[:,"tanshou_odds_at10_num"] = data.loc[:,"tanshou_odds_at10"].astype(float)
data.loc[:,"IDM_num"] = data.loc[:,"IDM"].astype(float)
data.loc[:,"hot_code_num"] = data.loc[:,"hot_code"].astype(float)

In [6]:
#人気の異常値をもつ行を削除
data =  data.loc[(data.loc[:,"popularity_num"]>=1) & (data.loc[:,"popularity_num"]<=18),:]


In [8]:
#探勝したかどうかをしめす列を追加
data.loc[:,"tanshou_flag"] = (data.loc[:,"order_of_finish_num"] == 1)


In [9]:
features = ["popularity_num","horse_weight_num","tanshou_odds_at10_num","IDM_num","hot_code_num"]
target = ["tanshou_flag"]

X = data
y = data.loc[:,target]
X = X.reset_index(drop=True)
y = y.reset_index(drop=True)
print("デザインマトリックスの形")
print(X.shape)
print("ラベルマトリックス")
print(y.shape)

デザインマトリックスの形
(28348, 21)
ラベルマトリックス
(28348, 1)


In [ ]:
np.set_printoptions(threshold=np.inf)

#レース単位で分割する
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=0)
#trainとtestにわける
for train, test in gss.split(X,y,groups=data["raceid"].values.flatten()):
    print( "%s %s" % (train,test))
    print( "%s %s" % (train.shape,test.shape))
    data_train = X.loc[train,:]
    data_test = X.loc[test,:]
    y_train = y.loc[train,:]
    y_test = y.loc[test,:]
    print(data_train.shape)
    print(data_test.shape)
    print(y_train.shape)
    print(y_test.shape)